In [2]:
import pandas as pd

In [3]:
import dvc.api

In [ ]:
# python main_inference_multimodal.py \
#         --model=LLM_Inference_Multimodal \
#         --model.llm_type="seqclf" \
#         --model.ckpt_path="models/product_title_multitask_multimodal/version_1/epoch=0-step=75000.ckpt" \
#         --model.config_path="models/product_title_multitask_multimodal/version_1/config.yaml" \
#         --model.task="seqclf_singlemodal_wishtitle2pseudov121tax" \
#         --model.output_dir="models/product_title_multitask_multimodal/version_1" \
#         --model.write_interval="batch" \
#         --data=JSONListData \
#         --data.llm_type="seqclf" \
#         --data.label_map_file="datasets/taxonomy/wish_v1.2.1_newtax_allpaths.txt" \
#         --data.label_type="taxonomy" \
#         --data.data_source_yaml_path="datasets/multimodal_multitask/wish_offshore_validated.yaml" \
#         --data.input_dict="{'template': '{title}', 'task_prefix': 'Classify product: '}" \
#         --data.output_dict="{'template': '{category}'}" \
#         --data.data_source_type="dvc" \
#         --data.model_name="t5-base" \
#         --data.batch_size=50 \
#         --data.max_length=50 \
#         --data.num_workers=0 \
#         --data.overwrite_cache=true \
#         --data.force_download_hfdata=true \
#         --trainer.logger=false \
#         --trainer.enable_checkpointing=false \
#         --trainer.accelerator gpu \
#         --trainer.strategy ddp

In [70]:
df_preds = pd.read_json('/workspaces/multitask-llm-rnd/modelling/models/product_title_multitask_multimodal/version_1/seqclf-epoch=0-step=75000--wish_offshore_validated--test.json', 
                        lines=True)

In [88]:
df_preds.head(2)

,preds,batch_idx,dataloader_idx,batch_indices,logits,top_pred
62,"[[consumer electronics, 0.17633756995201103], ...",0,0,0,"[-10.456609725952148, -10.014580726623535, -13...",entertainment > musical instruments > stringed...
45,"[[entertainment, 0.848908901214599], [entertai...",0,0,1,"[-11.307157516479492, -10.211978912353516, -13...",entertainment > musical instruments > stringed...


In [89]:
len(df_preds)

1024

In [67]:
df_test = pd.read_json(dvc.api.get_url( 
    'datasets/data/wish_products_internallabel/wish_products_offshore_labelled_validated_processed.json',
    repo='git@github.com:ContextLogic/multitask-llm-rnd.git'
), lines=True)

In [5]:
# df_test = pd.read_json(dvc.api.get_url( 
#     'datasets/data/liang_product/liang_product_classification_offshorev2only_test.json',
#     repo='git@github.com:ContextLogic/multitask-llm-rnd.git'
# ), lines=True)

In [90]:
df_test.head(2)

,pid,title,text,category,lance_predicted_category,category_path,category_pred
0,53d7e25dbcfb7338b24fddd4,3w/5w Electric Guitar Amplifier,3w/5w Electric Guitar Amplifier -> [entertainm...,"[entertainment, musical instruments, stringed ...","[entertainment, musical instruments, stringed ...",entertainment > musical instruments > stringed...,"[jewelry & accessories, jewelry making, beads]"
1,548a3d9f9f88e91005750ef1,New 3/5/7/9 Multiple Guitar Bass Stand Holder ...,New 3/5/7/9 Multiple Guitar Bass Stand Holder ...,"[entertainment, musical instruments, stringed ...","[entertainment, musical instruments, stringed ...",entertainment > musical instruments > stringed...,"[luggage & bags, coin purses & holders, key wa..."


In [78]:
df_test['category_path'] = df_test.category.apply(lambda x: ' > '.join(x))

In [68]:
len(df_test)

1024

In [98]:
df_preds = df_preds.sort_values('batch_indices').reset_index()

In [7]:
df_tax = pd.read_json(dvc.api.get_url(
    'datasets/data/taxonomy/wish_newtax.json',
    repo='git@github.com:ContextLogic/multitask-llm-rnd.git'
), lines=True)

In [8]:
df_tax_leaf = df_tax[df_tax.is_leaf]

In [9]:
valid_leaf_paths = set(df_tax_leaf['category_path'].str.lower())

In [74]:
df_preds['preds'].apply(lambda x: x[0][0] in valid_leaf_paths).all()

False

In [75]:
def get_top_leaf_pred(x):
    for i in x:
        if i[0] in valid_leaf_paths:
            return i[0]
    return 'unknown'

In [99]:
df_preds['top_pred'] = df_preds['preds'].apply(get_top_leaf_pred)

In [100]:
df_test.category_path.str.lower().apply(lambda x: x in valid_leaf_paths).all()

True

In [101]:
df_preds.top_pred.str.lower().apply(lambda x: x in valid_leaf_paths).all()

False

In [106]:
df_preds[~df_preds.top_pred.str.lower().apply(lambda x: x in valid_leaf_paths)].head(2)

,index,preds,batch_idx,dataloader_idx,batch_indices,logits,top_pred
103,131,"[[home improvement, 0.9015701413154601], [home...",0,0,103,"[-11.108057022094727, -10.607271194458008, -13...",unknown
137,150,"[[home improvement, 0.6993829607963561], [home...",0,0,137,"[-10.193086624145508, -9.886104583740234, -12....",unknown


In [80]:
from eval_from_file import perf_eval_util


In [107]:
df_test['category'] = df_test['category_path'].str.lower().apply(lambda x: x.split(' > '))
df_test['category_pred'] = df_preds['top_pred'].str.lower().apply(lambda x: x.split(' > '))

In [110]:
perf_eval_util(df_test, 1, 'category_pred')

,id,precision,recall,f1-score,support,level
0,apparel accessories,0.826087,0.558824,0.666667,34.000000,1
1,automobiles & motorcycles,0.791667,0.558824,0.655172,34.000000,1
2,beauty & health,0.851064,0.952381,0.898876,42.000000,1
3,cellphones & telecommunications,0.730159,0.938776,0.821429,49.000000,1
4,computer & office,0.916667,0.868421,0.891892,38.000000,1
5,consumer electronics,0.647059,0.717391,0.680412,46.000000,1
6,education & office supplies,0.705882,0.648649,0.676056,37.000000,1
7,electronic components & supplies,1.000000,0.470588,0.640000,17.000000,1
8,entertainment,0.953488,0.872340,0.911111,47.000000,1
9,food,0.000000,0.000000,0.000000,7.000000,1
